In [22]:
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import time
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import numpy as np
import os

In [2]:
model = Word2Vec.load("300features_40minwords_10context")

In [3]:
# 데이터 로드
train = pd.read_csv( "data/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )
test = pd.read_csv( "data/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "data/unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )

# 데이터 행개수 확인 (총 100,000 개)
print (("Read {0} labeled train reviews, {1} labeled test reviews, "+ '\n' +"and {2} unlabeled reviews").format(train["review"].size, test["review"].size, unlabeled_train["review"].size))

Read 25000 labeled train reviews, 25000 labeled test reviews, 
and 50000 unlabeled reviews


In [6]:
start = time.time() # Start time

# K를 vocabulary 길이의 1/5 로 클러스터수 설정, 즉 클러스터당 5개 단어들의 평균
word_vectors = model.wv.syn0
num_clusters = int(word_vectors.shape[0]/5)

# 중심점을 추출하기 위해, K-means 객체를 초기화
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# 훈련시간 측정
end = time.time()
elapsed = end - start
print ("Time taken for K Means clustering: "+ str(elapsed) + "seconds.")

Time taken for K Means clustering: 396.7486095428467seconds.


In [7]:
word_vectors.shape

(16490, 300)

In [9]:
int(word_vectors.shape[0]/5)

3298

In [10]:
# 단어 / 인덱스 사전 생성, 각 vocab에 있는 단어를 클러스터 번호에 맵핑                                        
word_centroid_map = dict(zip( model.wv.index2word, idx ))

In [11]:
# 첫 10개의 클러스터들에 대해서
for cluster in range(0,10):
    
    # 클러스터 번호 출력
    print ("\nCluster %d" % cluster)
    
    # 모든 단어들의 해당 클러스터(1~10)번호를 찾고 출력
    words = []
    for i in range(0,len(word_centroid_map.values())):        # 클러스터 번호 수 만큼 루프문
        if(list(word_centroid_map.values())[i] == cluster):   # 해당 클러스터가 1~10 차례의 클러스터 번호와 동일할 때
            words.append(list(word_centroid_map.keys())[i])   # 해당 위치의 단어를 1~10 클러스터 순으로 붙임
    print (words)                                             # 결국 각 단어들을 1~10번대의 클러스터 순으로 재정렬


Cluster 0
['enthusiast', 'fanatic', 'vegetarian', 'snob', 'masochist', 'believer']

Cluster 1
['wolfe', 'sheen', 'julian', 'mr', 'morris']

Cluster 2
['team', 'unit', 'ops']

Cluster 3
['apathetic', 'unreasonable']

Cluster 4
['deceit', 'greed', 'primal']

Cluster 5
['diego', 'miguel', 'flamboyant']

Cluster 6
['spirituality', 'masculinity', 'diatribe', 'accusations', 'degradation', 'strain', 'harshness', 'perversion', 'depravity', 'anarchy', 'manipulation', 'cruelty', 'unspeakable', 'pretension', 'ramblings', 'awakening', 'indifference', 'bondage', 'delusion', 'baggage']

Cluster 7
['skull', 'mask', 'bat', 'cap', 'worm']

Cluster 8
['conquest', 'celebration', 'antichrist', 'creation']

Cluster 9
['accuses', 'confessed', 'assumes', 'admits', 'proposal', 'admitting', 'recognizes', 'succumbs', 'denies', 'declares', 'reacts', 'fakes', 'responds', 'swears', 'argues', 'confronts', 'confesses', 'imagines', 'resigned', 'listens', 'expresses']


In [12]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    
    # 클러스터 개수는 사전에서 가장 높은 클러스터의 인덱스와 같다.
    num_centroids = max( word_centroid_map.values() ) + 1  # 0부터 시작하니 1을 더해줌
    
    # 속도를 위해 bag of centroids 벡터에 0을 사전할당 
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    
    # 리뷰에서 단어들을 루프 돌린다. 단어가 vocab에 있다면, 이 단어가 속한 클러스터를 찾아 빈도 1을 더함.
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    
    # 반환 값은 클러스터 빈도벡터
    return bag_of_centroids

In [13]:
def review_to_wordlist( review, remove_stopwords=False ):
    
    # document를 list화
    # 불용어를 없애는 것은 선택, 보통 높은 질의 워드벡터를 얻기 위해 불용어를 제거하지는 않음.
    
    # HTML 함수 지우기
    review_text = BeautifulSoup(review).get_text()
      
    # 글자가 아닌것들 없애기 []안의 ^는 not과 같음.
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
    # 소문자화 후 띄어쓰기를 기준으로 모두 나눔.
    words = review_text.lower().split()
    
    # 불용어 제거
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    
    # 단어 리스트 반환
    return(words)

In [24]:
# 평균을 얻는데 있어서 불용어는 단지 노이즈로 인식되기 때문에 불용어를 삭제한다.
clean_train_reviews = []

for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist(review, remove_stopwords=True))

clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist(review, remove_stopwords=True))

print ("Creating cleaned reviews")

D:\python\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 170 of the file D:\python\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Creating cleaned reviews


In [21]:
# 속도를 위해 bag of centroids에 0을 미리 할당.
# train셋에 있는 리뷰들을 bag of centroid로 변환

train_centroids = np.zeros((train["review"].size, num_clusters), dtype="float32")

counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids(review, word_centroid_map)
    counter += 1

# test셋에 대해서도 반복
test_centroids = np.zeros((test["review"].size, num_clusters), dtype="float32")

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids(review, word_centroid_map)
    counter += 1

In [23]:
# 랜덤포레스트 훈련 및 예측
forest = RandomForestClassifier(n_estimators = 100)

print ("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

# 테스트셋 결과 csv 파일로 저장

output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


In [25]:
output.head()

,id,sentiment
0,"""12311_10""",1
1,"""8348_2""",0
2,"""5828_4""",1
3,"""7186_2""",1
4,"""12128_7""",1
